In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity
import random
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk.stem.lancaster import LancasterStemmer
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
stemmer = LancasterStemmer()
from flask import Flask, render_template, request

C:\Users\Abc\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def cleanup(sentence):
    word_tok = nltk.word_tokenize(sentence)
    stemmed_words = [stemmer.stem(w) for w in word_tok]

    return ' '.join(stemmed_words)

In [3]:
le = LE()
tfv = TfidfVectorizer(min_df=1, stop_words='english')
data = pd.read_csv('BankFAQs.csv')
questions = data['Question'].values
X = []
for question in questions:
    X.append(cleanup(question))
tfv.fit(X)
le.fit(data['Class'])
X = tfv.transform(X)
y = le.transform(data['Class'])
trainx, testx, trainy, testy = tts(X, y, test_size=.25, random_state=42)
model = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=15),
    n_estimators=2000
)
model.fit(trainx, trainy)
print("Adaboost:", model.score(testx, testy))
    

Adaboost: 0.8600451467268623


In [4]:
def chat(usr):
    if usr.lower() == 'yes':
        #print("Bot: Yes!")
        return "YES!"
    if usr.lower() == 'no':
        #print("Bot: No?")
        return "No?"

    t_usr = tfv.transform([cleanup(usr.strip().lower())])
    class_ = le.inverse_transform(model.predict(t_usr)[0])
    questionset = data[data['Class']==class_]
    cos_sims = []
    for question in questionset['Question']:
        sims = cosine_similarity(tfv.transform([question]), t_usr)
        cos_sims.append(sims)
        
    ind = cos_sims.index(max(cos_sims))

    return data['Answer'][questionset.index[ind]]

In [5]:
print("PRESS Q to QUIT")
print()
print()
print("Bot: Hi, Welcome to our bank!")
while True:
    usr = input("You: ")
    if(usr=='Q'):
        print("Bot: It was good to help you.")
        break
    else:
        print("Bot:",chat(usr))

PRESS Q to QUIT


Bot: Hi, Welcome to our bank!
You: Hi
Bot: Hi how may I help you?
You: Q
Bot: It was good to help you.


In [6]:
app = Flask(__name__)
@app.route("/")
def home():
    return render_template("home.html")

In [7]:
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return str(chat(userText))

In [8]:
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Nov/2019 00:55:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 00:56:08] "GET /get?msg=Hi HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 00:56:24] "GET /get?msg=I%20want%20to%20open%20a%20back%20account HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 00:56:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 00:56:37] "GET /get?msg=Hi HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 00:56:57] "GET /get?msg=I%20want%20to%20open%20a%20bank%20account HTTP/1.1" 200 -
